In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import pearsonr
import statsmodels.api as smy
from scipy.stats import mannwhitneyu

In [ ]:
df_scPDSI = pd.read_csv('annual_scPDSI.csv')
df_pre = pd.read_csv('annual_pre.csv')
df_temp = pd.read_csv('annual_temp.csv')
df_tmax = pd.read_csv('annual_Tmax.csv')
df_tmin = pd.read_csv('annual_Tmin.csv')
df_dtr = pd.read_csv('annual_dtr.csv')
df_pet = pd.read_csv('annual_pet.csv')
df_sample = pd.read_csv('sample_site.csv')

In [ ]:
def get_dry_wet_years(df):
    dry_years = {}
    wet_years = {}

    for site in df.index:
        dry_years[site] = []
        wet_years[site] = []

        for year in df.columns:
            value = df.loc[site, year]

            if -4 < value < -0.5:
                dry_years[site].append(year)
            elif 0.5 < value < 4:
                wet_years[site].append(year)

    return dry_years, wet_years

dry_years, wet_years = get_dry_wet_years(df_scPDSI.dropna().set_index('site'))
def intersection_list_ordered(list1, list2):
    result = [value for value in list1 if value in list2]

    return result

list1 = [1, 2, 3, 4, 5]
list2 = [3, 4, 5, 6, 7]

result = intersection_list_ordered(list1, list2)
print(result)
def remove_adjacent_years(years):
    years.sort()
    result = []
    i = 0
    while i < len(years):
        if i < len(years) - 1 and years[i + 1] - years[i] <= 2:
            while i < len(years) - 1 and years[i + 1] - years[i] <= 2:
                i += 1
        else:
            result.append(years[i])
        i += 1
    
    return result
def calculate_confidence_interval(df, column):
    lower_bound = np.percentile(df[column], 2.5)
    upper_bound = np.percentile(df[column], 97.5)
    df_95 = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    mean_value = np.mean(df_95[column])
    print("95% Confidence Interval ({}): [{:.4f}, {:.4f}]".format(column, lower_bound, upper_bound))
    print("Mean ({}): {:.4f}".format(column, mean_value))
    return df_95

In [ ]:
list_Resistance = []
list_Recovery = []
list_site = []
from tqdm import tqdm
site_list =pd.merge(df_sample,df_scPDSI.dropna().reset_index(),on = 'site')['site'].to_list()
for i in tqdm(range(len(site_list)), desc="Calculating Growth_loss"):
    site_i = site_list[i]
    std_path = 'E:\experiment\DTR_AI_growth\data\ITRDB\FZS_STD_crn\\'+ site_i + '.rwlcrn.csv'
    df_rwl_i =pd.read_csv(std_path)
    dry_years_i  = [int(year) for year in dry_years[site_i]]
    dry_years_i_intersection = intersection_list_ordered(dry_years_i,df_rwl_i.index.to_list())
    dry_years2 = remove_adjacent_years(dry_years_i_intersection)
    
    wet_years_i = [int(year) for year in wet_years[site_i]]
    wet_years_i_intersection = intersection_list_ordered(wet_years_i,df_rwl_i.index.to_list())
    RWI_n = df_rwl_i.loc[wet_years_i_intersection,'std'].mean()

    dry_years2_Pre = [element - 2 for element in dry_years2]
    dry_years2_Pre_intersection = intersection_list_ordered(dry_years2_Pre,df_rwl_i.index.to_list())
    dry_years2_pos = [element + 2 for element in dry_years2]
    dry_years2_pos_intersection = intersection_list_ordered(dry_years2_pos,df_rwl_i.index.to_list())

    RWI_D = df_rwl_i.loc[dry_years2,'std'].mean()
    
    RWI_n = df_rwl_i.loc[wet_years_i_intersection,'std'].mean()#no-drought
    RWI_Pre = df_rwl_i.loc[dry_years2_Pre_intersection,'std'].mean()
    RWI_Post = df_rwl_i.loc[dry_years2_pos_intersection, 'std'].mean()
    if RWI_n > RWI_D:
        
        Resistance = RWI_D /RWI_Pre
        Recovery = RWI_Post / RWI_D
        
        list_Resistance.append(Resistance)
        list_Recovery.append(Recovery)
        list_site.append(site_i)
df_rr = pd.DataFrame({'site':list_site,'Resistance':list_Resistance
                     ,'Recovery':list_Recovery})
cs1 = pd.merge(df_rr,df_sample,on = 'site').dropna()
import seaborn as sns
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
grouped = cs1.groupby('AI_Class')
anova_results_dr = {}
anova_results_wr = {}
group_humid = grouped.get_group('Humid')
group_arid = grouped.get_group('Arid')
group_arid95 = calculate_confidence_interval(group_arid, 'Recovery')
group_humid95 = calculate_confidence_interval(group_humid, 'Recovery')
df_merge95 = pd.concat([group_arid95,group_humid95],axis=0)
f_statistic_dr, p_value_dr = f_oneway(group_arid95['Recovery'],group_humid95['Recovery'])
anova_results_dr['Recovery'] = {'F-statistic': f_statistic_dr, 'p-value': p_value_dr}
for key, value in anova_results_dr.items():
    print(f"{key}: F-statistic: {value['F-statistic']}, p-value: {value['p-value']}")
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
colors = [(255/255, 0/255, 0/255),(0/255, 92/255, 230/255)]
plt.figure(figsize=(5,4.5),dpi=100)
#palette=['#8DCDD5','#E6846D']
alpha = 0.5 
sns.boxplot(x="AI_Class", y="Recovery", data=df_merge95
            ,palette=colors,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(1, color='black', linestyle='--',linewidth=.6)
plt.ylim(0.5,2.5)
plt.yticks([0.5,1,1.5,2.0,2.5]) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('Resilience',fontsize=18)
plt.text(0.5, 1.8, '***', ha='center', va='center')
plt.text('Humid', 0.55,  f'n = {len(group_humid95)}', ha='center', va='center')
plt.text('Arid', 0.55,  f'n = {len(group_arid95)}', ha='center', va='center')
plt.savefig('Fig3a.jpg',dpi=300, bbox_inches='tight')
plt.show()
df_species = pd.read_excel('40tree.xlsx')
species_dict = dict(zip(df_species['Tree_Speci'], df_species['Clade']))
df_merge95['Species_group'] = df_merge95['Tree_Speci'].map(species_dict)
df_merge95['Species_group'] = df_merge95['Species_group'].fillna('Unknown')
df_merge95['log_aiv3']= np.log10(df_merge95['ai_v3'])
df_merge95['log_aiv3'].replace(-np.inf, np.nan, inplace=True)
df_merge95.dropna(subset=['log_aiv3'], inplace=True)


plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14
plt.figure(figsize=(5, 4.5))

species_groups = df_merge95['Species_group'].unique()
color_map = {
    species_groups[0]: (0/255, 92/255, 230/255),
    species_groups[1]: (255/255, 0/255, 0/255)
}

for species in species_groups:
    df_group = df_merge95[df_merge95['Species_group'] == species]

    r, p = pearsonr(df_group['log_aiv3'], df_group['Recovery'])
    print(r)
    X = df_group[['log_aiv3']]
    y = df_group['Recovery']
    X_with_const = sm.add_constant(X)  
    model = sm.OLS(y, X_with_const).fit()
    intercept, slope = model.params
    predictions = model.get_prediction(X_with_const)
    predicted_mean = predictions.predicted_mean
    conf_int = predictions.conf_int()

    plt.scatter(df_group['log_aiv3'], df_group['Recovery'], 
                alpha=0.08, label=f'{species} (r={r:.2f}, p={p:.2e})',
                color=color_map[species])

    plt.plot(df_group['log_aiv3'], predicted_mean, color=color_map[species], linestyle='-')

    plt.fill_between(df_group['log_aiv3'], conf_int[:, 0], conf_int[:, 1], color=color_map[species], alpha=0.2)
plt.ylim(0.5,2.5)
plt.yticks([0.5,1, 1.5, 2,2.5])
plt.xlabel('Log10(Aridity index)')
plt.ylabel('Resilience')
plt.legend(fontsize=12,loc=1)
plt.grid(True)
plt.savefig('Fig3b.jpg', dpi=300, bbox_inches='tight')
plt.show()

df_biome = pd.read_csv('gl_Biome.csv')
df95Recovery = pd.merge(df_biome,df_merge95.loc[:,('Recovery','site')],on='site')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.figure(figsize=(12, 6))
alpha = 0.7 
colors = [(0/255, 92/255, 230/255),(255/255, 0/255, 0/255)]
sns.boxplot(x='Biome_Re', y='Recovery', hue='AI_Class', data=df95Recovery, palette=colors
           ,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))

biomes = df95Recovery['Biome_Re'].unique()
for biome in biomes:
    if biome == 'Deserts':
        continue 
    
    dry = df95Recovery[(df95Recovery['Biome_Re'] == biome) & (df95Recovery['AI_Class'] == 'Arid')]['Recovery']
    wet = df95Recovery[(df95Recovery['Biome_Re'] == biome) & (df95Recovery['AI_Class'] == 'Humid')]['Recovery']

    t_stat, p_val = stats.ttest_ind(dry, wet, nan_policy='omit', equal_var=False)
    print(biome)
    print(t_stat)
    print(p_val)

    x1, x2 = biomes.tolist().index(biome) - 0.2, biomes.tolist().index(biome) + 0.2
    y, h, col = 0.6, 0.02, 'k' 
    
    if np.isfinite(y):
        plt.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.5, c=col)
        significance = ''
        if p_val < 0.001:
            significance = '***'
        elif p_val < 0.01:
            significance = '**'
        elif p_val < 0.05:
            significance = '*'
        
        plt.text((x1 + x2) * .5, y + h, f'{significance}', ha='center', va='bottom', color=col)
    n_dry = len(dry)
    n_wet = len(wet)
    plt.text(biomes.tolist().index(biome)- 0.25, y - 0.1, f'n = {n_wet}', ha='center', va='center', color='blue')
    plt.text(biomes.tolist().index(biome)+ 0.25 , y - 0.1, f'n = {n_dry}', ha='center', va='center', color='red')
    
plt.xticks(rotation=45)
plt.xlabel('')
plt.ylim(0.3,2.5)
plt.yticks([0.5,1.0,1.5,2.0,2.5])
plt.ylabel('Resilience')
plt.legend(title='Region')
plt.savefig('Fig3c.jpg', dpi=300, bbox_inches='tight')
plt.show()
df95Recovery['Latitude_bin'] = pd.cut(df95Recovery['Latitude'], bins=np.arange(-90, 91, 1))
latitude_stats = df95Recovery.groupby('Latitude_bin')['Recovery'].agg(['mean', 'sem']).reset_index()
slope, intercept, r_value, p_value, std_err = linregress(latitude_stats.dropna()['mean']
                                                         , latitude_stats.dropna()['Latitude_bin'].apply(lambda x: x.mid))
fig, ax = plt.subplots(figsize=(3, 6))
ax.axvline(x=1, color='gray', linestyle='--')  # 添加零值虚线
ax.plot(latitude_stats['mean'], latitude_stats['Latitude_bin'].apply(lambda x: x.mid), marker='o', linestyle='-', color='black')
ax.fill_betweenx(latitude_stats['Latitude_bin'].apply(lambda x: x.mid), 
                 latitude_stats['mean'] - 1.96 * latitude_stats['sem'], 
                 latitude_stats['mean'] + 1.96 * latitude_stats['sem'], 
                 color='orange', alpha=0.3)
ax.plot(latitude_stats.dropna()['mean'], intercept + slope * latitude_stats.dropna()['mean'],color='blue')
ax.set_xlabel('Resilience')
ax.set_ylabel('Latitude (°)')
ax.text(0.05, 0.35, f'r ={r_value:.2f}\np ={p_value:.2e}', transform=ax.transAxes
        , fontsize=14, verticalalignment='top')
plt.ylim(15,75)
ax.grid(True)
plt.savefig('FigS5b.jpg',dpi=600, bbox_inches='tight')
plt.show()
df95Recovery.to_csv('Recovery95.csv',index=False)
Gymnosperms = df95Recovery[df95Recovery['Species_group']=='Gymnosperms']
Angiosperms = df95Recovery[df95Recovery['Species_group']=='Angiosperms']
f_statistic_dr, p_value_dr = f_oneway(Gymnosperms['Recovery'], Angiosperms['Recovery'])
print(p_value_dr)
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 16
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
colors = [(255/255, 0/255, 0/255),(0/255, 92/255, 230/255)]
plt.figure(figsize=(4,3.5),dpi=100)
#palette=['#8DCDD5','#E6846D']
alpha = 0.5 
sns.boxplot(x="Species_group", y="Recovery", data=df95Recovery
            ,palette=colors,  fill=False, gap=.2, boxprops=dict(alpha=alpha)
            , whiskerprops=dict(alpha=alpha)
            , capprops=dict(alpha=alpha), medianprops=dict(alpha=alpha))
plt.axhline(1, color='black', linestyle='--',linewidth=.6)
plt.ylim(0.5,2.5)
plt.yticks([0.5,1,1.5,2.0,2.5]) 
plt.legend().set_visible(False)
plt.xlabel('')
plt.ylabel('Resilience',fontsize=18)
plt.text(0.5, 0.6, '**', ha='center', va='center')
plt.text('Gymnosperms', 0.6,  f'n= {len(Gymnosperms)}', ha='center', va='center')
plt.text('Angiosperms', 0.6,  f'n= {len(Angiosperms)}', ha='center', va='center')
plt.savefig('FigS4b.jpg',dpi=300, bbox_inches='tight')
plt.show()